In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
PATH = "data/dogbreed/"

In [4]:
sz = 256
arch = inception_4
bs = 32
# Jeremy tried 224, 58 and 299, 58

In [5]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

In [6]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.pivot_table(index='breed', aggfunc=len).sort_values('id', ascending=False)

In [7]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                        val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

## Precompute

In [8]:
data = get_data(sz, bs)

A Jupyter Widget

In [9]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 324/324 [09:49<00:00,  1.82s/it]


In [10]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.57257  0.35123  0.89725]                        
[ 1.       0.34734  0.31508  0.90904]                        



In [11]:
learn.fit(1e-2, 3)

A Jupyter Widget

[ 0.       0.32661  0.3451   0.90025]                        
[ 1.       0.28812  0.32767  0.90611]                        
[ 2.       0.26774  0.33788  0.90213]                        



## Augment

In [12]:
from sklearn import metrics

In [13]:
data = get_data(sz, bs)

A Jupyter Widget

In [14]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [15]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.59595  0.35948  0.89439]                        
[ 1.       0.45381  0.31755  0.91392]                        



In [16]:
learn.precompute=False
learn.bn_freeze=True

In [17]:
learn.fit(1e-2, 5, cycle_len=1)

A Jupyter Widget

[ 0.       0.4598   0.3023   0.91295]                        
[ 1.       0.41537  0.29842  0.91051]                        
[ 2.       0.40455  0.29762  0.91239]                        
[ 3.       0.36196  0.29892  0.91532]                        
[ 4.       0.38753  0.29721  0.91232]                        



In [18]:
learn.save('224_i4_pre')

In [19]:
learn.load('224_i4_pre')

## Increase size

In [20]:
learn.set_data(get_data(350, bs))
learn.freeze()

In [21]:
learn.fit(1e-2, 3, cycle_len=1)

A Jupyter Widget

[ 0.       0.25646  0.18973  0.93931]                        
[ 1.       0.27735  0.18744  0.93883]                        
[ 2.       0.27542  0.1914   0.94078]                        



In [22]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.25417  0.1885   0.93987]                        
[ 1.       0.25311  0.18667  0.94127]                        
[ 2.       0.22901  0.18404  0.94371]                        
[ 3.       0.2595   0.19297  0.93883]                        
[ 4.       0.23229  0.19491  0.94127]                        
[ 5.       0.21112  0.18988  0.9458 ]                        
[ 6.       0.21313  0.18727  0.94475]                        



In [23]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds, y), metrics.log_loss(y, probs)

(0.94667318982387472, 0.18084703138519545)

In [24]:
learn.save('350_i4_pre')

In [25]:
log_preds, y = learn.TTA(is_test=True)
probs_i4 = np.exp(log_preds)
#accuracy(log_preds, y), metrics.log_loss(y, probs)

In [26]:
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()

In [27]:
save_array('probs_i4.bc', probs_i4)

In [31]:
learn.fit(1e-2, 1, cycle_len=2)

A Jupyter Widget

[ 0.       0.23825  0.19766  0.94224]                        
[ 1.       0.21457  0.19238  0.94329]                        



In [32]:
learn.save('350_i4_pre')

In [33]:
log_preds, y = learn.TTA(is_test=True)
probs_i4_new = np.exp(log_preds)

In [34]:
save_array('probs_i4_new.bc', probs_i4_new)

## Use whole dataset now

In [31]:
learn.load('299_pre')

In [32]:
def get_data_whole(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                        val_idxs=[0], suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

In [33]:
def load_array(fname): return bcolz.open(fname)[:]

In [34]:
data = get_data_whole(sz, bs)

A Jupyter Widget

In [35]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.freeze()

100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


In [36]:
learn.fit(1e-2, 5)

A Jupyter Widget

[ 0.       0.75786  0.01884  1.     ]                         
[ 1.       0.38308  0.00202  1.     ]                         
[ 2.       0.27529  0.00073  1.     ]                         
[ 3.       0.22052  0.00052  1.     ]                         
[ 4.       0.19112  0.00022  1.     ]                         



In [37]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [38]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.94042  0.00524  1.     ]                         
[ 1.       0.48962  0.00265  1.     ]                         



In [39]:
learn.precompute = False
learn.bn_freeze = True

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.44712  0.00127  1.     ]                        
[ 1.       0.39743  0.00121  1.     ]                        
[ 2.       0.34848  0.00075  1.     ]                        
[ 3.       0.35622  0.00137  1.     ]                        
[ 4.       0.31755  0.00057  1.     ]                        
[ 5.       0.28701  0.00074  1.     ]                        
 76%|███████▌  | 134/177 [03:41<01:10,  1.65s/it, loss=0.29] 

In [ ]:
learn.set_data(get_data_whole(299, bs))
learn.freeze()

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA(is_test=True)
probs_resnx101_64_full = np.exp(log_preds)
#accuracy(log_preds, y), metrics.log_loss(y, probs)

In [ ]:
save_array('probs_resnx101_64_full.bc', probs_resnx101_64_full)

In [ ]:
learn.save('299_full')

## Try ensembling
### Use another arch first

## Save to file for submission

In [28]:
df = pd.DataFrame(probs_i4)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [29]:
df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,fd1a7be32f10493735555e62913c0841,3.974444e-07,7.507788e-07,9.347087e-07,2.096762e-06,3.142399e-07,3.856987e-07,4.041833e-07,2.873388e-07,1.197660e-07,...,8.481701e-07,3.980210e-07,1.059033e-06,1.114673e-06,3.316816e-07,5.090215e-06,1.412341e-07,8.128983e-08,1.642197e-07,2.543012e-08
1,5273d42f02b4715cb93845205995ef64,1.212713e-05,2.145150e-05,8.131643e-06,7.966507e-07,1.633836e-06,9.363218e-07,1.449693e-06,2.885816e-06,3.928386e-06,...,9.694937e-07,1.541736e-07,1.446600e-06,3.051312e-06,9.111903e-07,1.559473e-06,1.036390e-06,4.096230e-05,1.873804e-06,7.331846e-06
2,442057a3142f4d75a1023db363e2fb54,1.630715e-07,3.867569e-07,4.926990e-08,1.716040e-07,5.994908e-08,7.319854e-07,2.337359e-08,1.591795e-07,4.759276e-06,...,4.526489e-08,5.867522e-07,3.961799e-06,1.614672e-05,5.283512e-07,5.016151e-08,8.377177e-08,2.800566e-07,9.045405e-08,2.084151e-08
3,fb03302b030afd122e0f4936c158f6c1,3.457793e-05,4.042400e-07,4.574003e-07,3.007011e-07,1.260734e-06,2.460710e-07,4.961888e-06,4.454171e-08,2.751831e-07,...,6.100341e-06,3.745551e-06,5.101672e-07,2.455359e-07,3.577626e-07,2.193915e-07,1.981498e-06,8.246526e-07,5.500128e-07,3.437342e-05
4,511ca6e3539192aa5c8747bb697e0525,3.396206e-06,4.146918e-07,4.814238e-06,1.006872e-06,7.150266e-06,2.264200e-06,5.335293e-06,1.827578e-06,8.606753e-07,...,2.801771e-05,7.279037e-06,7.559770e-07,4.078439e-07,1.172226e-06,3.254182e-06,4.308861e-05,2.153625e-06,4.244859e-05,5.307658e-05


In [30]:
SUBM = f'{PATH}results_2/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}submission8.csv', index=False)
#df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]

In [ ]:
fn

In [ ]:
Image.open(PATH+fn).resize((150, 150))

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

In [ ]:
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
im = trn_tfms(Image.open(PATH+fn))
preds = to_np(learn.model(V(T(im[None]).cude())))
np.argmax(preds)

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

In [ ]:
im = trn_tfms(Image.open(PATH+fn))
preds = learn.predict_array(im[None])
np.argmax(preds)

## Confusion matrix

In [ ]:
preds = np.argmax(log_preds, axis=1)
probs = np.exp(log_preds[:,1])

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)